In [1]:
lang="en"
tokenized=True
remote=True

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 24 15:51:31 2016

@author: hedi & arame
"""
from collections import Counter
#import seaborn as sns
import numpy as np
import sys

def indexize(s, w2i,tokenized=True):
    res = []
    if len(s)==0:
        return res
    if s[0]=="#":
        try:
            return [w2i[s]]
        except:
            return res
    if tokenized:
        s=s.split(" ")
    for w in s:
        try:
            i = w2i[w]
        except:
            continue
        res.append(i)
    return res
    
class SentenceIt(object):
    def __init__(self, path):
        self.path = path
        self.n_data = 0
        
    def __iter__(self):
        for s in open(path,'r'):
            s = u"%s" % s.decode('utf-8')[:-1]
            if len(s)>0:
                self.n_data += 1
                yield s
            
class MinibatchSentenceIt(object):
    def __init__(self, path, batch_size, w2i,tokenized):
        self.path = path
        self.batch_size = batch_size
        self.w2i = w2i
        self.n_empty = 0
        self.i=0
        self.tokenized=tokenized
        
    def __iter__(self):
        # we could make it better by:
            # - taking overlapping batches
            # - randomly sampling batches
        minibatch = []
        Ls = []
        for s in open(path,'r'):
            s = u"%s" % s.decode('utf-8')[:-1]
            s = indexize(s, self.w2i,self.tokenized)
            if len(s) == 0:
                self.n_empty += 1
                continue
            if s[0]=="#":
                l=1
            else:
                l=len(s)
            minibatch.append(s)
            
            Ls.append(l)
            # Here it's savage, maybe find sth smarter and quicker
            if len(minibatch)==self.batch_size:
                self.i=self.i+1
                M = max(Ls)
                padded = np.array([np.pad(m, (0,M-l), 'constant') for (m,l) in zip(minibatch, Ls)], dtype='int32')                
                minibatch = []
                Ls = []
                yield padded
    
if __name__ == '__main__':
        
    strExtract=""
    strToken="_tok" if (tokenized and lang=="zh") else ""
    max_voc=20000
    if lang=="zh":
        path = '/media/data/datasets/wikipedia/entities/bigpage_zh'+strToken+'.txt_line_processed'+strExtract if remote else "data/dataset"+strToken+".txt"
    elif lang=="en":
        path='/media/data/datasets/wikipedia/entities/bigpage_'+lang+'.txt_line_processed'
        
    vocab = Counter()
    print "build vocab"
    Ls = []

    sentences = SentenceIt(path)
    if lang=="zh":
        for s in sentences:
            if s[0]=="#":
                Ls.append(1)
                vocab[s]+=1
            else:
                if tokenized:
                    s=s.split(" ")
                Ls.append(len(s))
                for w in s:
                    vocab[w] += 1
    else:
        for s in sentences:
            if tokenized:
                s=s.split(" ")
                Ls.append(len(s))
                for w in s:
                    vocab[w] += 1
            else:
                print "error: case not possible yet"
                
    #sns.distplot(Ls)
    n_data = sentences.n_data
    print "data size: " + str(n_data)
    w2i = {}
    i2cf = []
    i2w = []
    f = open('vocab','w')
    mc =  vocab.most_common()[:max_voc]
    cumFreq=0
    

    for k,(w,c) in enumerate([('<pad>', 0)] + mc):
        cumFreq+=pow(int(c),3./4)
        w2i[w] = k
        i2cf.append(cumFreq)
        i2w.append(w)

        f.write("%s %f\n" % (w.encode('utf-8'),int(c))) 
    i2f=[cf/float(cumFreq)for cf in i2cf]

    f.close()

    words=w2i.keys()
    words=map(lambda w:w[0],mc)
    print "vocab size: " + str(len(words))
    print "words: "+ str(words[:10])
    print "i2f: "+ str(i2f[:10])

    batch_size = 300 if remote else 5
    vocab_size = len(i2w)
    n_epochs = 500000 if remote else 2
    dim=200 if remote else 6
    pt="_pt" if lang!="en" else ""
    
    save_every = 1000 if remote else 4

    batches = MinibatchSentenceIt(path, batch_size, w2i,tokenized)
    
    print "begin"


    i2e={}
    index_fixe=[0]
    if lang=="zh":
        pretrainedFile="/media/data/datasets/models/word2vec_model/model_bridge/model_zh_ws5_pt_ne5_sa0.0001_mc40.vec" if remote else "data/pretrained.txt"
    elif lang=="en":
        pretrainedFile="/media/data/datasets/models/word2vec_model/model_bridge/model_en_ws5_ne5_sa0.0001_mc100.vec"
        

    if remote:
        sys.path.insert(0, '/home/arame/hakken-api/models/')
        import model
        import utils
        pretrained=model.model(pretrainedFile,max_voc=max_voc)
        wordsModel=pretrained.words
        floatsModel=pretrained.floats
    else:
        import utils
        wordsModel,floatsModel=utils.loadModel(pretrainedFile)
    
    print "finish load pretrain"
     
    for word,oldi in wordsModel.items(): 
        if word in words:
            i=w2i[word]
            if len(word)>1 and word[0:2]=="##":
                index_fixe.append(i)
            i2e[i]=floatsModel[oldi]
    
    print "len index_fixe:" + str(len(index_fixe))
    print "index_fixe: "+ str(index_fixe[:10])
    
    print "create model"
    lr=0.0025 if remote else 0.01
    neg_len=800 if remote else 10
    strNeg="_neg"+str(neg_len)
    strBs="_bs"+str(batch_size)

    saving_path = "/media/data/datasets/models/word2vec_model/model_fastsent/pickle_" + lang + pt + "_fastsent" + strToken + strBs + strNeg + ".vec" + strExtract if remote else "chineseModel"


build vocab
data size: 3
vocab size: 54
words: [u'saxophone', u'flat', u'soprano', u'alto', u'walking', u'#195727#', u'#73233#', u'#314843#', u'revised', u'labeled']
i2f: [0.0, 0.048872319603449614, 0.07793197470300424, 0.10699162980255886, 0.13605128490211346, 0.1533302592040712, 0.17060923350602894, 0.18788820780798665, 0.2051671821099444, 0.22244615641190213]
begin
processed: 0
finish load pretrain
len index_fixe:2
index_fixe: [0, 48]
create model


/usr/lib/python2.7/dist-packages/scipy/lib/_util.py:35: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/numpy/lib/utils.py:95: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [3]:
import theano
import numpy as np
from time import time
import theano.tensor as T
import cPickle
from theano.tensor.nnet import softmax
#dtype = theano.config.dtype
from random import random
from bisect import bisect

def weighted_choice(i2f):
    x = random()
    i = bisect(i2f, x)
    return i
dtype='float32'

class Model(object):
    @classmethod
    def load(cls,path):
        with open(path,'rb') as f:
            params= cPickle.load(f)
        w2i,W, V, autoencode= params
        return cls(W, V,w2i, autoencode)
    
    @classmethod
    def create(cls, vocab_size, dim,w2i,i2e={}, autoencode=True,cst=0.001):
        W = np.vstack((np.zeros(dim),0.01*np.random.randn(vocab_size-1, dim))).astype(dtype)
        V = np.vstack((np.zeros(dim),0.01*np.random.randn(vocab_size-1, dim))).astype(dtype)
        for i,e in i2e.items():
            W[i]=cst*e
            V[i]=cst*e
        return cls(W,V,w2i,autoencode)   

    def save(self, saving_path):
        print saving_path
        data = [self.w2i,self.W, self.V, self.autoencode]
        with open(saving_path,'wb') as f:
            cPickle.dump(data, f)
        return None

In [57]:
class FastSent(Model):
    def __init__(self, W,V,w2i,autoencode):
        self.w2i=w2i
        self.W = theano.shared(W, name='W')
        self.V = theano.shared(V, name='V')
        self.autoencode = autoencode
        self.params = [self.W,self.V]
        
        indexes = T.imatrix('X')
        batch_mask = 1-T.eq(indexes, T.zeros_like(indexes))
        mask = batch_mask[1:-1]
        maskp = batch_mask[:-2]
        maskn = batch_mask[2:]
        X_indexes = indexes[1:-1]
        Yp_indexes = indexes[:-2]
        Yn_indexes = indexes[2:]

        X = self.W[X_indexes]
        Xs = T.sum(X, axis=1)
        activations = T.dot(Xs, self.V.T)
        # Change this thing so we don't compute softmax for <pad> token
        prediction = softmax(activations)
        I = [Yp_indexes, Yn_indexes]
        M = [maskp, maskn]
        if self.autoencode:
            I.append(X_indexes)
            M.append(mask)
        y_true = T.concatenate(I, axis=1)
        mask = T.concatenate(M, axis=1)
        output = 1e-6+prediction[T.arange(y_true.shape[0]).reshape((-1,1)), 
                            y_true]
        
        cost = T.mean(T.sum(-T.log(output)*mask, axis=1))
        lr = T.scalar('lr', dtype=dtype)
        
        self.grads = T.grad(cost, self.params)
        # Here is sgd, we could make it better
        updates = [(p, p-lr*g) for p,g in zip(self.params, self.grads)]
        self._train = theano.function(inputs = [indexes, lr],
                                      outputs = [cost,activations,prediction,output,self.grads[1]], 
                                      #updates=updates, 
                                      allow_input_downcast=True,
                                      on_unused_input='warn')
                                     
                                      
    def train(self, batch_iterator, lr, min_lr, n_epochs, 
              saving_path, save_every, verbose = True):
        n_iter = 0
        break_all = False
        for epoch in xrange(n_epochs+1):
            learning_rate = min_lr + (n_epochs-epoch)*(lr-min_lr)/n_epochs
            for batch in batch_iterator:
                b = batch
                tic = time()
                print "norm V: " + str(np.linalg.norm(self.V.get_value()))
                cost,activations,prediction,output,grads1 = self._train(b, learning_rate)
                print np.max(activations)
                for i in activations:
                    print i,
                print ""
                for i in prediction:
                    print i,
                print ""
                print np.sum(prediction)
                print np.argmax(prediction)
                for i in output:
                    print i,
                toc = time() - tic
                n_iter += 1
                if not n_iter%save_every:
                    if verbose:
                        print "\tSaving model"
                    self.save(saving_path)
                if n_iter==n_epochs:
                    break_all = True
                if verbose:
                    print "Epoch %d Update %d Cost %f Time %fs" % (epoch,
                                                                   n_iter, 
                                                                   cost, 
                                                                   toc)
                if np.isnan(grads1).any():
                    print "stop because of grads1"
                if np.isnan(cost):
                    break_all = True
                if break_all:
                    break
            if break_all:
                break

In [58]:
batches = MinibatchSentenceIt(path, 3, w2i,tokenized)

In [59]:
model1 = FastSent.create(vocab_size, dim,w2i=w2i,i2e=i2e,cst=1)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: lr.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.


In [60]:
model1.train(batches, 
            lr=lr, 
            min_lr=lr/float(100), 
            n_epochs=1, 
            saving_path=saving_path, 
            save_every=save_every, 
            verbose=True)

norm V: 17.385
307.247
[  0.00000000e+00   2.26943573e+02   6.09856224e+01   1.64802948e+02
   1.74409424e+02   2.14155655e+01   1.32498190e-01  -5.51832557e-01
  -3.18536282e-01   3.77494278e+01   2.26036301e+01   1.54559600e+00
   2.59831200e+01  -1.12183258e-01   3.13861161e-01  -3.66684198e-01
   6.50928574e+01  -1.67141449e+00   2.66910980e+02  -3.33939821e-01
  -1.53493226e-01   2.73550957e-01   9.64015884e+01   3.56405716e+01
   7.24593341e-01  -9.09662992e-02   3.24163399e+01   5.54148778e-02
   3.20624466e+01   4.82878268e-01   5.08219779e-01   3.65635757e+01
  -8.01636755e-01   7.68050461e+01  -1.81703299e-01   2.69702835e+01
   3.00479946e+01   6.11308632e+01   9.52913463e-01   2.57305725e+02
  -3.01163197e-01   1.29099622e-01  -3.84017676e-01  -7.29373470e-02
   3.35973829e-01   3.07246674e+02   6.39380157e-01   3.08792152e+01
   2.63733521e+02   2.55853252e+01   4.17190313e-01   2.63426483e-01
   3.48912805e-01   2.91370773e+01   3.57279599e-01] 
[  0.00000000e+00   1.3329

In [24]:
import math
math.log(math.exp(-1.37853)/math.exp(307.985))

/usr/local/lib/python2.7/dist-packages/traitlets/traitlets.py:463: DeprecationWarning: _[traitname]_default handlers are deprecated: use default decorator instead
  dynamic_default = self._dynamic_default_callable(obj)
/usr/local/lib/python2.7/dist-packages/traitlets/traitlets.py:981: DeprecationWarning: _[traitname]_changed handlers are deprecated: use observe and unobserve instead
  'type': 'change',
/usr/local/lib/python2.7/dist-packages/traitlets/traitlets.py:451: DeprecationWarning: _[traitname]_default handlers are deprecated: use default decorator instead
  if (self._dynamic_default_callable(obj) is None) \


-309.36353

In [ ]:
model2 = FastSent.create(vocab_size, dim,w2i=w2i,i2e=i2e,cst=0.001)


In [ ]:
model2.train(batches, 
            lr=lr, 
            min_lr=lr/float(100), 
            n_epochs=10, 
            saving_path=saving_path, 
            save_every=save_every, 
            verbose=True)

In [ ]:
model.train(batches, 
            lr=lr, 
            min_lr=lr/float(100), 
            n_epochs=20, 
            saving_path=saving_path, 
            save_every=save_every, 
            verbose=True)

In [ ]:
modelNoInit = FastSent.create(vocab_size, dim,w2i=w2i)


In [ ]:
modelNoInit.train(batches, 
            lr=lr, 
            min_lr=lr/float(100), 
            n_epochs=5, 
            saving_path=saving_path, 
            save_every=save_every, 
            verbose=True)